In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PreprocessingConfig:
    root_dir: Path
    ingestion_dir: Path
    output_dir: Path

In [6]:

from pathlib import Path
from dataclasses import dataclass
import yaml
from sentimentanalyzer.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from sentimentanalyzer.utils.common import create_directories,read_yaml

In [7]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

  

    def get_preprocessing_config(self) -> PreprocessingConfig:
        config = self.config.data_preprocessing

        create_directories([config.root_dir])

        data_preprocessing_config = PreprocessingConfig(
                      root_dir =config.root_dir,
                      ingestion_dir = config.ingestion_dir,
                      output_dir = config.output_dir
               )
        
        return data_preprocessing_config

In [8]:
import pandas as pd
from pathlib import Path
from src.sentimentanalyzer.utils.common import convert_to_csv, preprocess_review_list, clean_sentiment_text
import re, contractions, emoji
from sentimentanalyzer.logging import logger


In [9]:



class DataPreprocessing:
    def __init__(self, config: PreprocessingConfig):
        self.config = config

        # Ensure output directory exists
        create_directories([Path(self.config.root_dir)])

    def preprocess(self):
        ingestion_dir = Path(self.config.ingestion_dir)
        output_dir    = Path(self.config.output_dir)

        # Find all .ft.txt files in ingestion_dir
        txt_files = list(ingestion_dir.glob("*.ft.txt"))
        if not txt_files:
            logger.warning(f"No .ft.txt files found in {ingestion_dir}")
            return

        for txt_path in txt_files:
            split_name = txt_path.stem.replace(".ft", "")  # e.g. 'train'
            csv_raw_path = output_dir / f"{split_name}_raw.csv"
            csv_cleaned  = output_dir / f"{split_name}_clean.csv"

            logger.info(f"Converting {txt_path.name} → {csv_raw_path.name}")
            convert_to_csv(txt_path, csv_raw_path)

            logger.info(f"Loading {csv_raw_path.name} into DataFrame")
            df_raw = pd.read_csv(csv_raw_path, low_memory=True)

            logger.info(f"Applying `preprocess_review_list` to {split_name}")
            df_clean = preprocess_review_list(df_raw)

            logger.info(f"Saving cleaned data to {csv_cleaned.name}")
            df_clean.to_csv(csv_cleaned, index=False)

        logger.info("Preprocessing complete.")


In [10]:

try:
    config = ConfigurationManager()
    data_preprocessing_config = config.get_preprocessing_config()
    data_preprocessor = DataPreprocessing(config=data_preprocessing_config)
    data_preprocessor.preprocess()
except Exception as e:
    raise e


[2025-06-15 00:05:48,070: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-15 00:05:48,080: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-15 00:05:48,082: INFO: common: created directory at: artifacts]
[2025-06-15 00:05:48,083: INFO: common: created directory at: artifacts/data_preprocessing]
[2025-06-15 00:05:48,084: INFO: common: created directory at: artifacts\data_preprocessing]
[2025-06-15 00:05:48,086: INFO: 2374303811: Converting test.ft.txt → test_raw.csv]
[2025-06-15 00:05:51,638: INFO: 2374303811: Loading test_raw.csv into DataFrame]
[2025-06-15 00:05:53,929: INFO: 2374303811: Applying `preprocess_review_list` to test]
[2025-06-15 00:06:24,955: INFO: 2374303811: Saving cleaned data to test_clean.csv]
[2025-06-15 00:06:30,984: INFO: 2374303811: Converting train.ft.txt → train_raw.csv]
[2025-06-15 00:07:14,845: INFO: 2374303811: Loading train_raw.csv into DataFrame]
[2025-06-15 00:07:30,726: INFO: 2374303811: Applying `preprocess_re